# Snippets book
useful or exploratory code that found no use but is yet too good to go...

# Stream muffler - filter stdout and stderr to take out annoying log items inserting themselves in the printout

In [ ]:
import sys
from contextlib import contextmanager


class StreamFilter:
    """Wraps a stream and filters out lines containing any of the strings in the provided exclude_list."""

    def __init__(self, stream, exclude_list):
        """Stream and a list of substrings to intercepted lines that should be excluded from the stream."""
        self.stream = stream
        self._buffer = ""
        self.exclude = exclude_list

    def write(self, message):
        """Write."""
        self._buffer += message
        while "\n" in self._buffer:
            line, self._buffer = self._buffer.split("\n", 1)
            line += "\n"
            if not any(sub in line for sub in self.exclude):
                self.stream.write(line)

    def flush(self):
        """Flush."""
        if self._buffer:
            if not any(sub in self._buffer for sub in self.exclude):
                self.stream.write(self._buffer)
            self._buffer = ""
        self.stream.flush()


@contextmanager
def muffler(exclude_list: list):
    """Context managaer that hijacks stdin and stdout to filter out noise.

    Lines containing any of the strings in exclude_list will be intercepted and ignored.
    """
    original_stdout = sys.stdout
    original_stderr = sys.stderr
    sys.stdout = StreamFilter(stream=original_stdout, exclude_list=exclude_list)
    sys.stderr = StreamFilter(stream=original_stderr, exclude_list=exclude_list)
    try:
        yield
    finally:
        sys.stdout.flush()
        sys.stderr.flush()
        sys.stdout = original_stdout
        sys.stderr = original_stderr


# usage:
exclude_list = ["WARN SparkSession: Using an existing Spark session"]
with muffler(exclude_list):
    print("1. next line will be a stupid warning from the spark engine")
    print("2. WARN SparkSession: Using an existing Spark session blabla bla")
    print("3. luckily that was filtered out so we don't see line #2")